In [1]:
import trimesh
import os
import numpy as np
from PIL import Image
os.environ["PYOPENGL_PLATFORM"] = "egl" #opengl seems to only work with TPU
import pyrender
from pyrender import IntrinsicsCamera,Mesh, Node, Scene,OffscreenRenderer
import cv2

In [2]:
# 1. load asset
resolver = trimesh.resolvers.FilePathResolver('mesh_material/UnicornHorn_OBJ/')
asset_mesh = trimesh.load(file_obj='mesh_material/UnicornHorn_OBJ/UnicornHorn.obj', resolver = resolver, process=False)

#mesh.vertices[:, 2] = mesh.vertices[:, 2] + 100.

color = asset_mesh.visual.to_color()
asset_mesh.visual.vertex_colors = color
img_size = 960

In [3]:
print(asset_mesh.visual.vertex_colors.vertex_colors)
print("color: {}".format(color))

[[255 255 255 255]
 [255 255 255 255]
 [255 255 255 255]
 ...
 [255 255 255 255]
 [255 255 255 255]
 [255 255 255 255]]
color: <trimesh.visual.color.ColorVisuals object at 0x7f5483bbd7f0>


In [4]:
# 2. load deformed fg mesh
#fg_mesh = trimesh.load('logdir/catamelie-dualrig002-properlyscaledcams-leftcam-ds-finetune-fgbkgd/obj0/catamelie-dualrig002-fgbg-leftcam-mesh-00000.obj', process=False)
fg_mesh = trimesh.load('logdir/human-dualrig002-properlyscaledcams-leftcam-ds-finetune-fgbkgd/obj0/human-dualrig-fgbg002-leftcam-mesh-00000.obj', process=False)

In [5]:
print("number of faces: {}".format(fg_mesh.faces.shape))
print("number of vertices: {}".format(fg_mesh.vertices.shape))
print("number of face normals: {}".format(fg_mesh.face_normals.shape))
print("number of vertex normals: {}".format(fg_mesh.vertex_normals.shape))

number of faces: (620776, 3)
number of vertices: (310236, 3)
number of face normals: (620776, 3)
number of vertex normals: (310236, 3)


In [6]:
####################################################
# 3. align (and scale) the 3d asset to the fg mesh
####################################################

# 3a. scale vertices of 3d asset
scale = 0.0005
asset_mesh.vertices = asset_mesh.vertices * scale

# 3b. transform vertices of 3d asset to fg frame using "asset2fg" transformation 
# 
#chosen_vertex = 55
chosen_vertex = 5831
asset2fg = np.eye(4)

# translation component = coordinate of the designated vertex in the fgframe
asset2fg[:3, 3] = fg_mesh.vertices[chosen_vertex, :].copy()
print("fg_mesh.vertices[chosen_vertex, :]: {}".format(fg_mesh.vertices[chosen_vertex, :]))

# rotation component:
# 2nd column: normal = fg_mesh.vertex_normal[chosen_vertex, :]
asset2fg[:3, 1] = fg_mesh.vertex_normals[chosen_vertex, :].copy()           # unit length normal vector

# 1st column: [-R_32, 0, R_12].T where 2nd column = [R_12, R_22, R_23].T
asset2fg[0, 0] = -asset2fg[2, 1]
asset2fg[1, 0] = 0.
asset2fg[2, 0] = asset2fg[0, 1]


asset_vertices_assetframe_homo = np.concatenate([asset_mesh.vertices, np.ones_like(asset_mesh.vertices[:, 0:1])], axis = -1)        # (N, 3) + (N, 1) = (N, 4)
asset_vertices_fgframe_homo = np.matmul(asset2fg, asset_vertices_assetframe_homo.T).T                                               # (4, N) = (4, 4) * (4, N) -> (N, 4)

fg_mesh.vertices[chosen_vertex, :]: [-0.01416462  0.12319944  0.02791327]


In [7]:

# 4a. load extrinsics (world2cam transformations)
#fg2cam = np.loadtxt('logdir/catamelie-dualrig002-properlyscaledcams-leftcam-ds-finetune-fgbkgd/obj0/catamelie-dualrig002-fgbg-leftcam-cam-00000.txt')
fg2cam = np.loadtxt('logdir/human-dualrig002-properlyscaledcams-leftcam-ds-finetune-fgbkgd/obj0/human-dualrig-fgbg002-leftcam-cam-00000.txt')
fg2cam[3, :] = np.array([0., 0., 0., 1.])

fg_vertices_fgframe_homo = np.concatenate([fg_mesh.vertices, np.ones_like(fg_mesh.vertices[:, 0:1])], axis = -1)                             # (N, 3) + (N, 1) = (N, 4)
fg_vertices_camframe_homo = np.matmul(fg2cam, fg_vertices_fgframe_homo.T).T                                                                  # (4, N) = (4, 4) * (4, N) -> (N, 4)

asset_vertices_camframe_homo = np.matmul(fg2cam, asset_vertices_fgframe_homo.T).T

asset_mesh.vertices = asset_vertices_camframe_homo[:, :3]               # (N, 3)
fg_mesh.vertices = fg_vertices_camframe_homo[:, :3]                     # (N, 3)

In [8]:
# 5. insert all meshes into the scene
r = OffscreenRenderer(img_size, img_size)
scene = Scene(ambient_light=0.4*np.asarray([1.,1.,1.,1.]))

asset_meshr = Mesh.from_trimesh(asset_mesh, smooth=True)
fg_meshr = Mesh.from_trimesh(fg_mesh, smooth=True)

scene.add_node(Node(mesh=asset_meshr))
scene.add_node(Node(mesh=fg_meshr))

In [9]:
# 6. add camera to the scene

cam_time = IntrinsicsCamera(
            800,
            800,
            360,
            480,
            znear=1e-3,zfar=1000)
cam_pose = -np.eye(4); cam_pose[0,0]=1; cam_pose[-1,-1]=1
cam_node = scene.add(cam_time, pose=cam_pose)

In [10]:
# 7. add a light source to the scene
theta = 9*np.pi/9
init_light_pose = np.asarray([[1,0,0,0],[0,np.cos(theta),-np.sin(theta),0],[0,np.sin(theta),np.cos(theta),0],[0,0,0,1]])
light_pose = init_light_pose
direc_l = pyrender.DirectionalLight(color=np.ones(3), intensity=6.0)
direc_l_node = scene.add(direc_l, pose=light_pose)

# 8. render Scene
mesh_rnd_color, mesh_rnd_depth = r.render(scene,flags=pyrender.RenderFlags.SHADOWS_DIRECTIONAL | pyrender.RenderFlags.SKIP_CULL_FACES)
r.delete()

#mesh_rnd_color = mesh_rnd_color[:960, :720, :3]
cv2.imwrite('mesh_material/UnicornHorn_OBJ/rendered_mesh.png', mesh_rnd_color[..., ::-1])

True

In [23]:
print("mesh_rnd_depth: {}".format(mesh_rnd_depth[mesh_rnd_depth > 0]))
print("mesh_rnd_color: {}".format(mesh_rnd_color))

mesh_rnd_depth: [0.31481358 0.31471983 0.314602   ... 0.2590786  0.2590376  0.25937387]
mesh_rnd_color: [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
